In [1]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
import math
from numpy import arange

from pandas import read_csv
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
import seaborn as sns
import pylab

from scipy import stats
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV


In [2]:
#THE CCT DATA
dataframe = pd.read_csv('CCT-TFS syyskuu Henry.csv')
dataframe
print(dataframe.shape)
#print(dataframe.head(n=4))
dataframe
dataframe.dropna(inplace=True)

(1272, 14)


In [3]:
#CENTERING THE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


new_dat=dataframe.drop(['Steel'], axis=1)
centered_dat=pd.DataFrame(scaler.fit_transform(new_dat), columns=new_dat.columns)
centered_dat.head()



,C,Si,Mn,Cr,Mo,Ni,Al,Cu,ppmB,Pa,Rc,lnRc,Texp
0,-0.981735,-0.426369,-1.296763,-0.783937,-0.800529,-0.477096,-0.197778,0.563468,-0.376174,0.052783,5.117114,2.340625,0.906620
1,-0.981735,-0.426369,-1.296763,-0.783937,-0.800529,-0.477096,-0.197778,0.563468,-0.376174,0.052783,0.496718,1.521658,1.653774
2,-0.981735,-0.426369,-1.296763,-0.783937,-0.800529,-0.477096,-0.197778,0.563468,-0.376174,0.052783,-0.207556,0.444309,2.080719
3,-0.981735,-0.426369,-1.314646,-0.783937,-0.800529,-0.477096,-0.197778,0.563468,-0.376174,0.052783,-0.261385,-0.630567,2.400928
4,-0.981735,-0.426369,-1.314646,-0.783937,-0.800529,-0.477096,-0.197778,0.563468,-0.376174,0.052783,-0.265512,-1.689007,2.721137


In [41]:
#NORMALIZING THE DATA
#dat=dataframe.drop(['Steel'],axis=1)
scaler=MinMaxScaler()
new_dat[new_dat.columns.values] = scaler.fit_transform(new_dat)
new_dat

newdat=new_dat=dataframe.drop(['Steel','Texp','C','Si','Mn','Cr','Mo','Ni','Al','Cu','ppmB'], axis=1)
newdat
#STANDARDIZED DATA; CENTERED(WITHOUT MEAN) + NORMALIZED (KEPT WITHIN A RANGE)

,Pa,Rc,lnRc
0,1132.36,60.000,4.094345
1,1132.36,8.500,2.140066
2,1132.36,0.650,-0.430783
3,1132.36,0.050,-2.995732
4,1132.36,0.004,-5.521461
...,...,...,...
1267,1286.59,6.000,1.791759
1268,1286.59,3.000,1.098612
1269,1286.59,1.500,0.405465
1270,1286.59,1.500,0.405465


In [5]:
#DEPENDENT AND INDEPENDENT VARIABLES
#x=Dependent Variables
#y=Independent Variables

data = pd.DataFrame.values
x=new_dat.drop(['Texp'], axis = 1)
y=new_dat.Texp
x=x.astype(float)
y=y.astype(float)


In [34]:
interaction=new_dat.drop(['Texp','lnRc','Rc','Pa'], axis = 1)
interaction.columns
col = interaction.columns
for i in col:
    #print(i)
    for j in col:
        interaction[i +' | '+j]= interaction[i]*interaction[j]
        #print(interaction)
        for k in col:
            interaction[i + '|' + j + '|' + k] = interaction[i] * interaction[j] * interaction[k]
            print(interaction)
            
#for k in col:
#    print(k)
#interaction

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C     C|C|C  
0     0.108374  

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...   C|Si|Cr   C|Si|Mo   C

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...   C|Cr|Cr   C|Cr|Mo   C

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...   C|Ni|Mo   C|Ni|Ni   C

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...   C|Cu|Ni   C|Cu|Al   C

...

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...  Cu|ppmB|Al  Cu|ppmB|Cu

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...  ppmB|Si|Al  ppmB|Si|Cu

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...  ppmB|Cr|Cu  ppmB|Cr|pp

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...  ppmB|Ni|ppmB  ppmB | A

             C        Si        Mn        Cr        Mo        Ni        Al  \
0     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
1     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
2     0.130952  0.102662  0.043738  0.014599  0.000000  0.011342  0.000000   
3     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
4     0.130952  0.102662  0.041750  0.014599  0.000000  0.011342  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
1267  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1268  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1269  0.214286  0.136882  0.159046  0.204380  0.007576  0.003781  0.023697   
1270  0.214286  0.152091  0.353877  0.049270  0.007576  0.217391  0.023697   
1271  0.214286  0.155894  0.361829  0.049270  0.469697  0.005671  0.023697   

            Cu      ppmB     C | C  ...  ppmB|Cu|Cu  ppmB|Cu|pp

In [47]:
interaction
interaction.to_csv('nth_interaction.csv', index = False)
#DropboxPeprah_PhyCodesINTERACTIONSOBJECTIVE_ONEOBJECTIVE_ONE


In [17]:
#for a in x.columns:
#    for b in x.columns:
#        x[a + '|' + b] = x[a] * x[b]
     #   for c in dat.columns:
     #       dat[a + '|' + b + '|' + c] = dat[a] * dat[b] * dat[c]

interaction
col

Index(['C', 'Si', 'Mn', 'Cr', 'Mo', 'Ni', 'Al', 'Cu', 'ppmB', 'ppmB | C',
       'ppmB | Si', 'ppmB | Mn', 'ppmB | Cr', 'ppmB | Mo', 'ppmB | Ni',
       'ppmB | Al', 'ppmB | Cu', 'ppmB | ppmB', 'ppmB | ppmB|C',
       'ppmB | ppmB|Si', 'ppmB | ppmB|Mn', 'ppmB | ppmB|Cr', 'ppmB | ppmB|Mo',
       'ppmB | ppmB|Ni', 'ppmB | ppmB|Al', 'ppmB | ppmB|Cu',
       'ppmB | ppmB|ppmB'],
      dtype='object')

In [ ]:
col = interaction.columns
for i in col:
    for j in col:
        interaction[i +' | '+j]= interaction[i]*interaction[j]
        for k in col:
            interaction[i +' | '+j+'|'+k]= interaction[i]*interaction[j]

In [45]:
interaction 

,C,Si,Mn,Cr,Mo,Ni,Al,Cu,ppmB,C | C,...,ppmB | ppmB,ppmB|ppmB|C,ppmB|ppmB|Si,ppmB|ppmB|Mn,ppmB|ppmB|Cr,ppmB|ppmB|Mo,ppmB|ppmB|Ni,ppmB|ppmB|Al,ppmB|ppmB|Cu,ppmB|ppmB|ppmB
0,0.130952,0.102662,0.043738,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.017149,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.130952,0.102662,0.043738,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.017149,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.130952,0.102662,0.043738,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.017149,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.130952,0.102662,0.041750,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.017149,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.130952,0.102662,0.041750,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.017149,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,0.214286,0.136882,0.159046,0.204380,0.007576,0.003781,0.023697,0.004926,0.886076,0.045918,...,0.785131,0.168242,0.107470,0.124872,0.160465,0.005948,0.002968,0.018605,0.003868,0.695685
1268,0.214286,0.136882,0.159046,0.204380,0.007576,0.003781,0.023697,0.004926,0.886076,0.045918,...,0.785131,0.168242,0.107470,0.124872,0.160465,0.005948,0.002968,0.018605,0.003868,0.695685
1269,0.214286,0.136882,0.159046,0.204380,0.007576,0.003781,0.023697,0.004926,0.886076,0.045918,...,0.785131,0.168242,0.107470,0.124872,0.160465,0.005948,0.002968,0.018605,0.003868,0.695685
1270,0.214286,0.152091,0.353877,0.049270,0.007576,0.217391,0.023697,0.004926,0.886076,0.045918,...,0.785131,0.168242,0.119411,0.277839,0.038683,0.005948,0.170681,0.018605,0.003868,0.695685
